In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 12432101
paper_name = 'deutschbauer_davis_2002' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [54]:
files = ['sporulation_deficient.txt','sporulation_proficient.txt','germination.txt']

In [55]:
original_data_list = []
for f in files:
    original_data = pd.read_csv('raw_data/' + f, sep='\t')
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    original_data['orf'] = original_data['Orf'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data.set_index('orf', inplace=True)
    
    if f.startswith('sporulation'):
        original_data = original_data[['Spo1','Spo2']].apply(pd.to_numeric, axis=1, errors='coerce')
        original_data[['Spo1','Spo2']] = 1/original_data[['Spo1','Spo2']]
    else:
        original_data = original_data[['Germ1','Germ2']].apply(pd.to_numeric, axis=1, errors='coerce')
        
    original_data['data'] = original_data.mean(axis=1)
    
    original_data = original_data[['data']].copy()
    
    original_data = original_data.groupby(original_data.index).mean()
    
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 261 x 11
        Gene        Orf   Spo1   Spo2  HomDip oligo microarray Under 15%  \
0       SWI4    YER111C  2.146  1.706   0.896   SK1                        
1      ISC10    YER180C  2.461  3.338   1.008  Both    induced             
2       NDI1    YML120C  1.904  4.154   0.994  W303                        
3       SMA2    YML066C  2.787  5.121   1.011  Both    induced       yes   
4  YML010W-A  YML010W-A  1.831  1.623   0.878                              

               Biological Process          Molecular Function  \
0                      cell cycle        transcription factor   
1               not yet annotated  molecular_function unknown   
2  complex I (NADH to ubiquinone)           not yet annotated   
3      biological_process unknown  molecular_function unknown   
4      biological_process unknown  molecular_function unknown   

                Cellular Component  
0                not yet annotated  
1                not yet annotated  
2  NAD

In [56]:
original_data1 = pd.concat(original_data_list[:2], axis=0)
original_data1 = original_data1.groupby(original_data1.index).mean()

In [57]:
original_data2 = original_data_list[2]

In [58]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [59]:
original_data.shape

(511, 2)

In [60]:
original_data.head()

,data_1,data_2
orf,,
YAL009W,0.481032,NaN
YAL011W,1.813767,NaN
YAL021C,0.299158,NaN
YAL023C,1.559497,NaN
YAL040C,3.653337,NaN


# Load & process tested strains

In [61]:
tested = pd.read_csv('raw_data/SpoGerm_RawData.txt', sep='\t')

In [62]:
tested.head()

,ORF,TAG,PreSpo1,Spore1,Germ1_0,Germ1_1,Germ1_2,Germ1_3,Germ1_4,PreSpo2,Spore2,Germ2_0,Germ2_1,Germ2_2,Germ2_3,HomDip1,HomDip2,HomDip3,HomDip4,HomDip5
0,YAL001C,tag_504_at,4493.5,3859.510,6054.720,9187.320,6446.8,7044.920,7094.82,8424.5,9522.99,8390.94,11803.5,12326.200,16568.000,17379.8,22187.20,15202.200,16804.300,11942.500
1,YAL001C,tag_504_st,1346.8,1584.370,2485.530,4449.700,3604.0,3409.650,4348.13,2632.0,4148.20,3655.08,3330.3,2992.700,4072.030,7049.5,3571.54,4541.910,4308.060,4049.180
2,YAL002W,tag_503_at,14272.5,14658.500,22995.900,16368.500,11318.5,10084.800,9199.27,16980.3,14116.00,12438.00,6537.8,4795.830,5004.240,19607.8,36820.10,20150.100,27186.100,19964.200
3,YAL002W,tag_503_st,15677.0,20186.900,31668.800,18967.400,14515.8,12260.700,13379.9,19562.5,20254.20,17846.50,4940.5,3395.100,4190.340,26168.3,24247.90,22050.400,27313.000,25009.100
4,YAL003W,tag_502_at,539.3,575.698,903.143,670.166,516.3,618.163,688.684,652.5,1577.45,1389.93,671.0,688.043,869.671,610.5,1073.07,592.109,896.593,724.973


In [63]:
tested['orf'] = tested['ORF     '].astype(str)

In [64]:
tested['orf'] = clean_orf(tested['orf'])

In [65]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [66]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [ORF     , TAG          , PreSpo1, Spore1, Germ1_0, Germ1_1, Germ1_2, Germ1_3, Germ1_4, PreSpo2, Spore2, Germ2_0, Germ2_1, Germ2_2, Germ2_3, HomDip1, HomDip2, HomDip3, HomDip4, HomDip5, orf]
Index: []

[0 rows x 21 columns]


In [67]:
tested['essential'] = is_essential(tested['orf'])

Loading SGD features from 2017-04-03.


In [68]:
tested = tested.loc[~tested['essential']]

In [69]:
tested.shape

(19184, 22)

In [70]:
tested_orfs = tested['orf'].unique()

In [71]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [72]:
# WT = 1
original_data = original_data.reindex(index=tested_orfs, fill_value=1)

In [73]:
original_data.shape

(4791, 2)

# Prepare the final dataset

In [74]:
data = original_data.copy()

In [75]:
dataset_ids = [478, 16004]
datasets = datasets.reindex(index=dataset_ids)

In [76]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [77]:
data.head()

dataset_id,478,16004
data_type,value,value
orf,,
YAL002W,1.0,1.0
YAL004W,1.0,1.0
YAL005C,1.0,1.0
YAL007C,1.0,1.0
YAL008W,1.0,1.0


## Subset to the genes currently in SGD

In [78]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [79]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,478,16004
,data_type,value,value
gene_id,orf,,
2,YAL002W,1.0,1.0
1863,YAL004W,1.0,1.0
4,YAL005C,1.0,1.0
5,YAL007C,1.0,1.0
6,YAL008W,1.0,1.0


# Normalize

In [80]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [81]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [82]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,478,16004,478,16004
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,1.0,1.0,-0.078737,0.0
1863,YAL004W,1.0,1.0,-0.078737,0.0
4,YAL005C,1.0,1.0,-0.078737,0.0
5,YAL007C,1.0,1.0,-0.078737,0.0
6,YAL008W,1.0,1.0,-0.078737,0.0


# Print out

In [83]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [52]:
from IO.save_data_to_db3 import *

In [53]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 12432101...
Inserting the new data...


100%|██████████| 2/2 [00:14<00:00,  7.07s/it]

Updating the data_modified_on field...
